In [15]:
#Importing necessary libraries
from sagemaker.huggingface import HuggingFace
import sagemaker
import os
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
import os

# Remove proxy vars if not needed
os.environ.pop("HTTP_PROXY", None)
os.environ.pop("HTTPS_PROXY", None)

# Use the certifi-provided CA bundle
os.environ["AWS_CA_BUNDLE"] = "/Volumes/LaCie/Projects_portfolio/NLP/SupportIQ/venv/lib/python3.11/site-packages/certifi/cacert.pem"

# Test AWS call
import boto3
print(boto3.client('sts').get_caller_identity())

{'UserId': 'AIDA2PNZRCZDH2V4XL47W', 'Account': '720332985926', 'Arn': 'arn:aws:iam::720332985926:user/dev-user', 'ResponseMetadata': {'RequestId': '55015a3c-ac1b-43c9-b62f-8dfe73913cae', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '55015a3c-ac1b-43c9-b62f-8dfe73913cae', 'content-type': 'text/xml', 'content-length': '405', 'date': 'Fri, 18 Jul 2025 04:23:24 GMT'}, 'RetryAttempts': 0}}


In [16]:
#settinng up AWS profile
os.environ['AWS_PROFILE'] = 'dev-user'

session = sagemaker.Session()

In [17]:
#Defining hyper parameters
hyper_parameters = {
    'model_id': 'google/flan-t5-base',
    'task_type':'seq2seq',
    'rank': 64,
    'alpha': 128,
    'dropout': 0.05,
    'bias': 'none',
    'lr': 1e-5,
    'epochs': 10,
    'wd': 0.01,
    'logging_steps': 50,
    'batch_size': 32,
    'save_steps': 200,
    'eval_steps': 100,
    'target_module': 'q,k,v',
    'early_stopping': 3,
    'max_length': 64
}

In [18]:
huggingface_estimator = HuggingFace(
            entry_point='train.py',
            source_dir=os.getenv("SOURCE_DIR"),
            role=os.getenv("ROLE"),
            instance_type='ml.g4dn.xlarge',
            instance_count=1,
            transformers_version='4.49.0',
            py_version='py311',
            pytorch_version='2.5.1',
            hyperparameters = hyper_parameters
)

In [ ]:
huggingface_estimator.fit({'training': 's3://gen-ai-repository/finetuning/flan-t5/data/'}, wait=False)

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2025-07-18-04-23-32-177


In [41]:
huggingface_estimator.latest_training_job.describe()["TrainingJobStatus"]

'InProgress'

In [26]:
job_name = huggingface_estimator.latest_training_job.name

In [34]:
sm = boto3.client('sagemaker')

In [51]:
status = sm.describe_training_job(TrainingJobName=job_name)
print("Status:", status['TrainingJobStatus'])

Status: InProgress


In [45]:
#After Job complete run this to see logs
# estimator = HuggingFace.attach(training_job_name=job_name)  
# estimator.logs(stream=True, wait=False)

In [ ]:
model_artifact_s3_uri = huggingface_estimator.model_data
print("Model Artifact S3 URI:")
print(model_artifact_s3_uri)